In [5]:
import os
from dotenv import load_dotenv
from agents import Agent, Runner, handoff , RunContextWrapper, TResponseInputItem, GuardrailFunctionOutput, OutputGuardrailTripwireTriggered, input_guardrail, output_guardrail
from pydantic import BaseModel

In [6]:
load_dotenv()

api_key=os.environ.get("OPENAI_API_KEY")

if not api_key:
    raise ValueError("Problem with API key")

In [7]:
@output_guardrail
async def forbidden_words_guardrail( ctx: RunContextWrapper, agent: Agent, output:str) ->GuardrailFunctionOutput:
    print(f"Checking output for forbidden words {output}")

    forbidden_phrases=["fart", "booger", "silly goose"]

    output_lower = output.lower()

    found_phrases = [phrase for phrase in forbidden_phrases if phrase in output_lower]
    trip_triggered = bool (found_phrases)

    print(f"Found forbidden phrase : {found_phrases}")

    return GuardrailFunctionOutput(
        tripwire_triggered=trip_triggered,
        output_info={
            "reason":"Output conatins forbidden phrases.",
            "forbiden phrase found": found_phrases,

        }
    )

agent= Agent(
    name="Customer Support Agent",
    instructions="You are a customer support agent, you help customers with their queries",
    output_guardrails=[forbidden_words_guardrail],
    model="gpt-4o-mini"
)

try:
    await Runner.run(agent, "Say the word fart")
    print("Guardrail didnt trip - this is unexpected")

except OutputGuardrailTripwireTriggered:
    print("The agent said a bad word. he is fired")

Checking output for forbidden words Fart! If you have any questions or need assistance, feel free to ask!
Found forbidden phrase : ['fart']
The agent said a bad word. he is fired
